In [ ]:
from tensorflow import keras
from keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras.utils import image_dataset_from_directory

## Load and prepare the dataset

In [ ]:
train_dataset = image_dataset_from_directory(
    directory = '/content/dataset/train',
    labels = 'inferred',
    label_mode = 'binary',
    batch_size = 64,
    image_size = (224,224)
)

test_dataset = image_dataset_from_directory(
    directory = '/content/dataset/test',
    labels = 'inferred',
    label_mode = 'binary',
    batch_size = 64,
    image_size = (224,224)
)

Found 8000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


## Normalize the images

In [ ]:
#normalize images to [0,1]
def norm_img(image, label):
    return image/255, label

train_dataset = train_dataset.map(norm_img)
test_dataset = test_dataset.map(norm_img)

## 4. Model 1: Transfer Learning with VGG16
### 4.1. Load Pre-trained VGG16 Base Model

In [ ]:
base_model = VGG16(weights='imagenet', include_top= False, input_shape=(224, 224, 3))


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

### 4.2. Build the Classification Head


In [ ]:
# Add custom layers on top of the base model
x = base_model.output
x = Flatten()(x)
x = Dense(128,activation='relu')(x)
x = Dense(1,activation='sigmoid')(x)

new_model = Model(inputs = base_model.input, outputs = x)

### 4.3. Compile the Model

In [ ]:
new_model.compile(optimizer=Adam(learning_rate=0.001), metrics= ['accuracy'], loss= 'binary_crossentropy')

### 4.4. Train the Model


In [ ]:
history = new_model.fit(train_dataset, validation_data= test_dataset, epochs= 5)

Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 70s 403ms/step - accuracy: 0.6226 - loss: 1.2450 - val_accuracy: 0.9105 - val_loss: 0.2216
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 61s 382ms/step - accuracy: 0.9226 - loss: 0.1979 - val_accuracy: 0.9285 - val_loss: 0.1848
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 85s 405ms/step - accuracy: 0.9396 - loss: 0.1541 - val_accuracy: 0.9320 - val_loss: 0.1722
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 81s 399ms/step - accuracy: 0.9633 - loss: 0.1103 - val_accuracy: 0.9270 - val_loss: 0.1738
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 49s 393ms/step - accuracy: 0.9736 - loss: 0.0838 - val_accuracy: 0.9280 - val_loss: 0.1831


### 4.5. Evaluate the Model

In [ ]:
_ , accuracy = new_model.evaluate(test_dataset)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

32/32 ━━━━━━━━━━━━━━━━━━━━ 10s 304ms/step - accuracy: 0.9272 - loss: 0.1912
Validation Accuracy: 92.80%


## 5. Model 2: Transfer Learning with ResNet50
### 5.1. Load Pre-trained ResNet50 Base Model

In [ ]:
base_model_resnet = ResNet50(include_top=False, weights= 'imagenet', input_shape=(224,224,3))

for layer in base_model_resnet.layers:
    layer.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


### 5.2. Build the Classification Head

In [ ]:
x = base_model_resnet.output

x = Flatten()(x)
x = Dense(128, activation = 'relu')(x)
x = Dense(1, activation= 'sigmoid')(x)

new_model2 = Model(inputs = base_model_resnet.input, outputs = x)

### 5.3. Compile the Model

In [ ]:
new_model2.compile(optimizer=Adam(learning_rate = 0.001), metrics = ['accuracy'], loss = 'binary_crossentropy')

### 5.4. Train the Model

In [ ]:
history = new_model2.fit(train_dataset, epochs = 15, validation_data= test_dataset )

Epoch 1/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 48s 264ms/step - accuracy: 0.5618 - loss: 1.8120 - val_accuracy: 0.6190 - val_loss: 0.6663
Epoch 2/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 71s 244ms/step - accuracy: 0.6210 - loss: 0.6610 - val_accuracy: 0.6315 - val_loss: 0.6549
Epoch 3/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 26s 206ms/step - accuracy: 0.6352 - loss: 0.6455 - val_accuracy: 0.6490 - val_loss: 0.6554
Epoch 4/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 45s 241ms/step - accuracy: 0.6316 - loss: 0.6483 - val_accuracy: 0.6605 - val_loss: 0.6361
Epoch 5/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 37s 209ms/step - accuracy: 0.6542 - loss: 0.6323 - val_accuracy: 0.6590 - val_loss: 0.6370
Epoch 6/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 40s 200ms/step - accuracy: 0.6631 - loss: 0.6222 - val_accuracy: 0.6720 - val_loss: 0.6201
Epoch 7/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 47s 245ms/step - accuracy: 0.6783 - loss: 0.6062 - val_accuracy: 0.6845 - val_loss: 0.6107
Epoch 8/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 26s 205ms/step - accuracy: 0.6872 - loss: 0

### 5.5 Evaluate the model

In [15]:
_ , accuracy = new_model2.evaluate(test_dataset)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

32/32 ━━━━━━━━━━━━━━━━━━━━ 5s 161ms/step - accuracy: 0.6695 - loss: 0.6579
Validation Accuracy: 67.60%
